In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import logging
import pandas as pd

In [2]:
# Set root logger level higher than DEBUG
logging.basicConfig(level=logging.DEBUG)  # DEBUG, INFO, WARNING, ERROR, CRITICAL

# Specifically silence noisy loggers
logging.getLogger("selenium").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

In [3]:
def start_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in background
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # If chromedriver is in PATH, you don’t need to specify it
    driver = webdriver.Chrome(options=chrome_options)

    return driver

In [4]:
# def scrape_data():
url = "https://ca.soccerway.com/national/france/ligue-1/20242025/regular-season/r81802/matches/"
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_experimental_option("detach", True)
driver = start_driver()
driver.get(url)

# Load all matches
for _ in range(40):
    try:
        # Load previous button
        btn = driver.find_element(By.XPATH, f"/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div/div[2]/div[1]/div/div/div/button")
        btn.click()
        time.sleep(2)
    except:
        break
    
# single match div
match = driver.find_element(By.XPATH, f"/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div/div[2]/div[2]/div/div/div[3]/div/div/div/a")
# return match

In [5]:
print(match.text)
print(match.get_attribute('outerHTML'))


<a class="sc-22ef6ec-0 sc-a1a6abf-2 boVFdS gAkJuB" href="/matches/2025/05/17/france/ligue-1/nantes/montpellier/4373049/"></a>


In [6]:
href = match.get_attribute('href')
print(href)

https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/nantes/montpellier/4373049/


In [7]:
matches = driver.find_elements(By.XPATH, "//a[contains(@href, '/match/')]")
print(matches)

[]


In [8]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(driver.page_source, 'html.parser')
all_links = soup.find_all('a', href=True)

matches = [a for a in all_links if '/matches/' and '/france/ligue-1' in a['href']]
matches

[<a class="sc-22ef6ec-0 boVFdS" href="/national/france/ligue-1/2025-2026/regular-season/2d58bc25-77ec-4425-bed7-30f1839e3f8f/" style="width: 100%;"><div class="sc-575aeb00-1 gTtAsv"><span class="sc-600113f9-0 bzoXQy"><span class="sc-600113f9-1 klwUeB"><img alt="" data-nimg="1" decoding="async" height="18" loading="lazy" src="https://static.soccerway.com/flags/svg/france.svg" style="color: transparent;" width="18"/></span></span><span class="sc-4e4c9eab-2 jgDxUJ label label">Ligue 1</span></div></a>,
 <a class="sc-22ef6ec-0 boVFdS" href="/national/france/ligue-1/20242025/regular-season/r81802/" style="width: 100%;"><div class="sc-94e50ec4-0 fxiSqU total"><div class="sc-94e50ec4-0 qzEgh sc-d7965a66-0 hxTmOI"><span class="sc-4e4c9eab-2 etMNoc label">Summary</span></div></div></a>,
 <a class="sc-22ef6ec-0 boVFdS" href="/national/france/ligue-1/20242025/regular-season/r81802/matches/" style="width: 100%;"><div class="sc-94e50ec4-0 fxiSqU total"><div class="sc-94e50ec4-0 qzEgh sc-d7965a66-0 

In [9]:
match_hrefs = set()
europe_ligues = ['france/ligue-1', 'germany/bundesliga']

def get_match_hrefs(years):
    all_hrefs = set()
    for year in years:
        for ligue in europe_ligues:
            match_links = driver.find_elements(By.CSS_SELECTOR, f"a[href*='{ligue}'][href*='/matches/{year}/']")
            hrefs = {a.get_attribute('href') for a in match_links}
            all_hrefs.update(hrefs)
    return all_hrefs

# Example: load matches for 2024 only
target_years = [2023, 2024, 2025]  # or [2023, 2024, 2025] if you want multiple
hrefs = get_match_hrefs(target_years)

match_hrefs.update(hrefs)

for href in hrefs:
    print(href)

print(f"Total unique matches collected: {len(match_hrefs)}")


https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/lens/monaco/4373050/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/lille/reims/4373047/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/nantes/montpellier/4373049/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/marseille/rennes/4373043/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/lyon/angers/4373044/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/strasbourg/le-havre/4373048/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/saint-tienne/toulouse/4373046/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/nice/brest/4373045/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/psg/auxerre/4373051/
Total unique matches collected: 9


In [10]:
print(match_hrefs)

{'https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/lens/monaco/4373050/', 'https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/lille/reims/4373047/', 'https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/nantes/montpellier/4373049/', 'https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/marseille/rennes/4373043/', 'https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/lyon/angers/4373044/', 'https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/strasbourg/le-havre/4373048/', 'https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/saint-tienne/toulouse/4373046/', 'https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/nice/brest/4373045/', 'https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/psg/auxerre/4373051/'}


In [11]:
def get_teams(driver):
    try:
        team_a = driver.find_element(By.XPATH, "//div[@class='team team-a']//a").text.strip()
        team_b = driver.find_element(By.XPATH, "//div[@class='team team-b']//a").text.strip()
        return f"{team_a} vs {team_b}"
    except:
        return "Unknown vs Unknown"

In [12]:
while True:
    try:
        consent_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "/html/body/div[8]/div[2]/div[2]/div[2]/button"))
            )
        consent_button.click()    # trigger consent button to remove modal
        print("✅ Consent button clicked.")
    except:
        pass    # consent may have been handled
        
    try:
        previous_count = len(match_hrefs)
        # remove the pop-up modal, cookie modal, consent button
        WebDriverWait(driver, 15).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, "fc-dialog-overlay"))
        )
        load_prev_btn = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div/div[2]/div[1]/div/div/div/button"))
        )
        load_prev_btn.click()   # trigger load previous button
        # driver.execute_script("arguments[0].click();", load_prev_btn)
        
        # Wait for new content to load
        time.sleep(4)  # Or better: wait for a new match element to appear
        # Get new hrefs
        new_hrefs = get_match_hrefs([2023, 2024, 2025])
        print(f"Loaded {len(new_hrefs - match_hrefs)} matches")
        match_hrefs.update(new_hrefs)
    except Exception as e:
        print("No more 'Load Previous' button or loading stopped:", e)
        break
    
print(f"Total matches found for 2025 in france/ligue-1: {len(match_hrefs)}")
for href in sorted(match_hrefs):
    print(href)

No more 'Load Previous' button or loading stopped: Message: 

Total matches found for 2025 in france/ligue-1: 9
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/lens/monaco/4373050/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/lille/reims/4373047/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/lyon/angers/4373044/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/marseille/rennes/4373043/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/nantes/montpellier/4373049/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/nice/brest/4373045/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/psg/auxerre/4373051/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/saint-tienne/toulouse/4373046/
https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/strasbourg/le-havre/4373048/


In [15]:
driver.get("https://ca.soccerway.com/matches/2025/05/17/france/ligue-1/strasbourg/le-havre/4373048/")
scored_minute = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[2]/div[2]")
# scored_minute = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[3]/div[2]")
# scored_minute = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[4]/div[2]")
# scored_minute = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[5]/div[2]")
# scored_minute = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[6]/div[2]/span/span")
print(scored_minute.text)
# print(type(scored_minute.text))

20'
1
0


In [26]:
xpaths = [
    "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[2]/div[2]/span/span",
    "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[3]/div[2]/span/span",
    "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[4]/div[2]/span/span",
    "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[5]/div[2]/span/span",
    "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[6]/div[2]/span/span"
]

# === Extract and print scored minutes ===
minutes_list = []
for xpath in xpaths:
    try:
        scored_minute = driver.find_element(By.XPATH, xpath)
        scored_minute_text = scored_minute.text.strip()
        # print([scored_minute_text])
        if scored_minute_text:
            minutes_list.append(scored_minute_text)
            print(minutes_list)
    except Exception as e:
        print(f"Could not find element at {xpath} - {e}")

["20'"]
["20'", "43'"]
["20'", "43'", "53'"]


In [24]:
type(scored_minute_text)

str

In [28]:
def parse_minute(minute_str):
    # Handle "90+3'" -> 93, "45'" -> 45
    minute_str = minute_str.replace("'", "").strip()
    if "+" in minute_str:
        base, extra = minute_str.split("+")
        return int(base) + int(extra)
    return int(minute_str)

parsed_minutes = [parse_minute(minute) for minute in minutes_list]
print(parsed_minutes)  # e.g., [20, 43, 53]

[20, 43, 53]


In [80]:
import re

valid_minutes = [entry for entry in scored_minute.text if re.search(r"Scored minute: \d+'", entry)]
valid_minutes

[]

In [76]:
valid_minutes = [entry for entry in scored_minute.text.split() if entry.isdigit() and 0 < int(entry) <= 70]
print("Valid minutes:", valid_minutes)

Valid minutes: []


In [48]:
def get_goal_minutes(goal_events_div_xpath):  # arg: goal_events_div_xpath
    try:
        team_div = driver.find_element(By.XPATH, goal_events_div_xpath)
        spans = team_div.find_elements(By.XPATH, ".//span")
        minutes = []
        for span in spans:
            text = span.text.strip()
            if "'" in text:
                try:
                    minute = int(text.replace("'", "").split('+')[0])
                    minutes.append(minute)
                except ValueError:
                    continue
        return minutes
    except NoSuchElementException:
        print(f"⚠️ No goal data found at {goal_events_div_xpath} for {driver.current_url}")
        return []

In [70]:
get_goal_minutes("/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[5]/div[2]")

[]

In [29]:
scored_minutes = []
index = 1

while True:
    try:
        xpath = f"(//div[contains(text(), 'Scored minute:')])[{index}]"
        element = driver.find_element(By.XPATH, xpath)
        text = element.text.strip()

        # Check if it's a valid minute using regex
        match = re.search(r"(\d+)'", text)
        if match:
            minute = int(match.group(1))
            scored_minutes.append(minute)
        else:
            # Blank or invalid format → end loop
            break

        index += 1
    except Exception:
        # Element not found or no more matches
        break

print("Valid Scored Minutes:", scored_minutes)

Valid Scored Minutes: []


In [ ]:
zero_zero_count = 0
late_goals_count = 0
valid_scores = {'0 - 1', '1 - 0', '1 - 1', '0 - 2', '2 - 0', '1 - 2', '2 - 1'}
late_goal_teams = []
late_goal_matches = []
goal_count = 0

for href in match_hrefs:
    driver.get(href)
    time.sleep(2)
    
    try:
        # Handle consent popup
        consent_button = driver.find_element(By.XPATH, "/html/body/div[8]/div[2]/div[2]/div[2]/button")
        consent_button.click()
        time.sleep(1)
    except:
        pass  # Consent may have already been handled

    try:
        home_score_elem = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div")
        away_score_elem = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/div/div/div[3]/div")
        home_team_elem = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/a[1]/div")
        away_team_elem = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/a[2]/div")

        home_score = int(home_score_elem.text.strip())
        away_score = int(away_score_elem.text.strip())
        score = f"{home_score} - {away_score}"
        total_goals = home_score + away_score

        if score == "0 - 0":
            zero_zero_count += 1
            print(zero_zero_count)
            continue
        
        if score in valid_scores and 1 <= total_goals <= 3:
            # home_goals_minutes = get_goal_minutes("/html/body/div[3]/div/div/div[1]/div/div[1]/div/div[5]/div[1]")
            # away_goals_minutes = get_goal_minutes("/html/body/div[3]/div/div/div[1]/div/div[1]/div/div[5]/div[2]")
            xpaths = [
                "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[2]/div[2]/span/span",
                "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[3]/div[2]/span/span",
                "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[4]/div[2]/span/span",
                "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[5]/div[2]/span/span",
                "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[6]/div[2]/span/span"
            ]

            # === Extract and print scored minutes ===
            minutes_list = []
            for xpath in xpaths:
                try:
                    scored_minute = driver.find_element(By.XPATH, xpath)
                    scored_minute_text = scored_minute.text.strip()
                    # print([scored_minute_text])
                    if not scored_minute_text:
                        break  # Stop if no more scored minutes
                    minutes_list.append(scored_minute_text)
                    print(minutes_list)
                except Exception as e:
                    print(f"Could not find element at {xpath} - {e}")
                    
            parsed_minutes = [parse_minute(minute) for minute in minutes_list]  # e.g., [20, 43, 53]
            # print(parsed_minutes)  # e.g., [20, 43, 53]
            
            for minute in parsed_minutes:
                if minute <= 70:
                    goal_count += 1
                
                    # Check if goal_count is valid and all goals are after 70th minute
                if goal_count and 1 <= minute <= 90:
                    late_goals_count += 1
                    late_goal_teams.append(f"{home_team_elem.text.strip()} {int(home_score_elem.text.strip())} vs {int(away_score_elem.text.strip())} {away_team_elem.text.strip()}")

            # all_goals = sorted(home_goals_minutes + away_goals_minutes)

            # if all_goals and all(minute >= 70 for minute in all_goals):
            #     late_goals_count += 1
            #     late_goal_teams.append(f"{home_team_elem.text.strip()} vs {away_team_elem.text.strip()}")
            
            late_goal_matches.append({
                "zero_zero_count": zero_zero_count,
                "late_goals_count": late_goals_count,
                "late_goal_teams": late_goal_teams,
                "goal_count": goal_count
            })
            
            df_zero = pd.DataFrame([{"0-0 Count": zero_zero_count}])
            df_late = pd.DataFrame(late_goal_matches)

            df_zero.to_csv("data/zero_zero_matches.csv", index=False)
            df_late.to_csv("data/late_goal_matches.csv", index=False)

    except Exception as e:
        print(f"Error processing {href}: {e}")
        continue

print(f"\n✅ Total 0-0 games: {zero_zero_count}")
print(f"✅ Total 1-3 goal games with first goal after 70th minute: {late_goals_count}")
print("✅ Matches where all goals were after 70th minute:")
for match in late_goal_teams:
    print(f" - {match}")


["37'"]
["37'", "60'"]
["37'", "60'", "86'"]
Could not find element at /html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[5]/div[2]/span/span - Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[5]/div[2]/span/span"}
  (Session info: chrome=138.0.7204.183); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
#0 0x55cd901972ca <unknown>
#1 0x55cd8fc3e550 <unknown>
#2 0x55cd8fc900f0 <unknown>
#3 0x55cd8fc902e1 <unknown>
#4 0x55cd8fcde5e4 <unknown>
#5 0x55cd8fcb5bed <unknown>
#6 0x55cd8fcdb9e6 <unknown>
#7 0x55cd8fcb5993 <unknown>
#8 0x55cd8fc81d6b <unknown>
#9 0x55cd8fc83141 <unknown>
#10 0x55cd9015c2ab <unknown>
#11 0x55cd901600b9 <unknown>
#12 0x55cd90143139 <unknown>
#13 0x55cd90160c68 <unknown>
#14 0x55cd9012

In [72]:
def extract_match_data(driver, match_hrefs):
    zero_zero_count = 0
    late_goals_count = 0
    valid_scores = {'0 - 1', '1 - 0', '1 - 1', '0 - 2', '2 - 0', '1 - 2', '2 - 1'}
    late_goal_teams = []
    late_goal_matches = []
    goal_count = 0

    for href in match_hrefs:
        driver.get(href)
        time.sleep(2)
        
        try:
            # Handle consent popup
            consent_button = driver.find_element(By.XPATH, "/html/body/div[8]/div[2]/div[2]/div[2]/button")
            consent_button.click()
            time.sleep(1)
        except:
            pass  # Consent may have already been handled

        try:
            home_score_elem = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div")
            away_score_elem = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/div/div/div[3]/div")
            home_team_elem = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/a[1]/div")
            away_team_elem = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/a[2]/div")

            home_score = int(home_score_elem.text.strip())
            away_score = int(away_score_elem.text.strip())
            score = f"{home_score} - {away_score}"
            total_goals = home_score + away_score

            if score == "0 - 0":
                zero_zero_count += 1
                print(zero_zero_count)
                continue
            
            if score in valid_scores and 1 <= total_goals <= 3:
                # home_goals_minutes = get_goal_minutes("/html/body/div[3]/div/div/div[1]/div/div[1]/div/div[5]/div[1]")
                # away_goals_minutes = get_goal_minutes("/html/body/div[3]/div/div/div[1]/div/div[1]/div/div[5]/div[2]")
                xpaths = [
                    "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[2]/div[2]/span/span",
                    "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[3]/div[2]/span/span",
                    "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[4]/div[2]/span/span",
                    "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[5]/div[2]/span/span",
                    "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[6]/div[2]/span/span"
                ]

                # === Extract and print scored minutes ===
                minutes_list = []
                for xpath in xpaths:
                    try:
                        scored_minute = driver.find_element(By.XPATH, xpath)
                        scored_minute_text = scored_minute.text.strip()
                        # print([scored_minute_text])
                        if not scored_minute_text:
                            break  # Stop if no more scored minutes
                        minutes_list.append(scored_minute_text)
                        print(minutes_list)
                    except Exception as e:
                        print(f"Could not find element at {xpath} - {e}")
                        
                parsed_minutes = [parse_minute(minute) for minute in minutes_list]  # e.g., [20, 43, 53]
                # print(parsed_minutes)  # e.g., [20, 43, 53]
                
                for minute in parsed_minutes:
                    if minute <= 70:
                        goal_count += 1
                    
                        # Check if goal_count is valid and all goals are after 70th minute
                    if goal_count and 1 <= minute <= 90:
                        late_goals_count += 1
                        late_goal_teams.append(f"{home_team_elem.text.strip()} {int(home_score_elem.text.strip())} vs {int(away_score_elem.text.strip())} {away_team_elem.text.strip()}")

                # all_goals = sorted(home_goals_minutes + away_goals_minutes)

                # if all_goals and all(minute >= 70 for minute in all_goals):
                #     late_goals_count += 1
                #     late_goal_teams.append(f"{home_team_elem.text.strip()} vs {away_team_elem.text.strip()}")
                late_goal_matches.append({
                    "zero_zero_count": zero_zero_count,
                    "late_goals_count": late_goals_count,
                    "late_goal_teams": late_goal_teams,
                    "goal_count": goal_count
                })
                
                df_zero = pd.DataFrame([{"0-0 Count": zero_zero_count}])
                df_late = pd.DataFrame(late_goal_matches)

                df_zero.to_csv("data/zero_zero_matches.csv", index=False)
                df_late.to_csv("data/late_goal_matches.csv", index=False)

        except Exception as e:
            print(f"Error processing {href}: {e}")
            continue
        
    return df_zero, df_late

In [73]:
extract_match_data(driver, match_hrefs)

["37'"]
["37'", "60'"]
["37'", "60'", "86'"]
Could not find element at /html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[5]/div[2]/span/span - Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[5]/div[2]/span/span"}
  (Session info: chrome=138.0.7204.183); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
#0 0x55cd901972ca <unknown>
#1 0x55cd8fc3e550 <unknown>
#2 0x55cd8fc900f0 <unknown>
#3 0x55cd8fc902e1 <unknown>
#4 0x55cd8fcde5e4 <unknown>
#5 0x55cd8fcb5bed <unknown>
#6 0x55cd8fcdb9e6 <unknown>
#7 0x55cd8fcb5993 <unknown>
#8 0x55cd8fc81d6b <unknown>
#9 0x55cd8fc83141 <unknown>
#10 0x55cd9015c2ab <unknown>
#11 0x55cd901600b9 <unknown>
#12 0x55cd90143139 <unknown>
#13 0x55cd90160c68 <unknown>
#14 0x55cd9012

(   0-0 Count
 0          0,
    zero_zero_count  late_goals_count  \
 0                0                 3   
 1                0                 5   
 
                                      late_goal_teams  goal_count  
 0  [Lille 2 vs 1 Reims, Lille 2 vs 1 Reims, Lille...           2  
 1  [Lille 2 vs 1 Reims, Lille 2 vs 1 Reims, Lille...           3  )

In [76]:
df_zero, df_late = extract_match_data(driver, match_hrefs)

print(df_late)

["37'"]
["37'", "60'"]
["37'", "60'", "86'"]
Could not find element at /html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[5]/div[2]/span/span - Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]/div[5]/div[2]/span/span"}
  (Session info: chrome=138.0.7204.183); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
#0 0x55cd901972ca <unknown>
#1 0x55cd8fc3e550 <unknown>
#2 0x55cd8fc900f0 <unknown>
#3 0x55cd8fc902e1 <unknown>
#4 0x55cd8fcde5e4 <unknown>
#5 0x55cd8fcb5bed <unknown>
#6 0x55cd8fcdb9e6 <unknown>
#7 0x55cd8fcb5993 <unknown>
#8 0x55cd8fc81d6b <unknown>
#9 0x55cd8fc83141 <unknown>
#10 0x55cd9015c2ab <unknown>
#11 0x55cd901600b9 <unknown>
#12 0x55cd90143139 <unknown>
#13 0x55cd90160c68 <unknown>
#14 0x55cd9012

In [51]:
def close_overlay_if_present():
    try:
        overlay = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, "fc-dialog-overlay"))
        )
        # overlay = driver.find_element(By.XPATH, "/html/body/div[8]/div[2]/div[2]/div[2]/button")
        driver.execute_script("arguments[0].remove();", overlay)
        print("Overlay removed.")
    except:
        pass  # Overlay not present

In [54]:
close_overlay_if_present()

In [58]:
def remove_all_overlays():
    driver.execute_script("""
        document.querySelectorAll('.fc-dialog-overlay, .fc-dialog-container').forEach(el => el.remove());
    """)
    print("All overlays removed.")
    
remove_all_overlays()

All overlays removed.


In [60]:
def handle_consent_popup():
    try:
        consent_popup = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, "fc-consent-root"))
        )
        driver.execute_script("arguments[0].remove();", consent_popup)
        print("Consent popup removed.")
    except:
        print("No consent popup found.")

handle_consent_popup()

Consent popup removed.


In [66]:
def remove_all_overlays():
    blocking_classes = [
        "fc-dialog-overlay",         # Consent
        "fc-consent-root",           # Cookie pop
        # "sc-94e50ec4-0",             # Custom site overlay
        # "sc-e15f478f-1",             # Another wrapper layer
    ]
    for cls in blocking_classes:
        try:
            overlays = driver.find_elements(By.CLASS_NAME, cls)
            for overlay in overlays:
                driver.execute_script("arguments[0].remove();", overlay)
                print(f"✅ Removed overlay: {cls}")
        except Exception as e:
            print(f"⚠️ Could not remove overlay {cls}: {e}")


In [69]:
def click_previous_button():
    
    handle_consent_popup()  # Ensure consent popup is handled
    remove_all_overlays()  # Ensure button isn't blocked

    try:
        prev_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div/div[2]/div[1]/div/div/div/button"))
        )
        # prev_btn = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div/div[2]/div[1]/div/div/div/button")
        driver.execute_script("arguments[0].scrollIntoView(true);", prev_btn)
        time.sleep(0.5)  # Ensure visibility
        prev_btn.click()
        time.sleep(2)  # Wait for data to load
        return True
    except Exception as e:
        print("No more previous matches or error occurred:", e)
        return False

# Usage
while True:
    success = click_previous_button()
    if not success:
        break
    print(extract_match_data(driver, match_hrefs))


No consent popup found.
No more previous matches or error occurred: Message: 
Stacktrace:
#0 0x55cd901972ca <unknown>
#1 0x55cd8fc3e550 <unknown>
#2 0x55cd8fc900f0 <unknown>
#3 0x55cd8fc902e1 <unknown>
#4 0x55cd8fcde5e4 <unknown>
#5 0x55cd8fcb5bed <unknown>
#6 0x55cd8fcdb9e6 <unknown>
#7 0x55cd8fcb5993 <unknown>
#8 0x55cd8fc81d6b <unknown>
#9 0x55cd8fc83141 <unknown>
#10 0x55cd9015c2ab <unknown>
#11 0x55cd901600b9 <unknown>
#12 0x55cd90143139 <unknown>
#13 0x55cd90160c68 <unknown>
#14 0x55cd9012760f <unknown>
#15 0x55cd901851f8 <unknown>
#16 0x55cd901853d6 <unknown>
#17 0x55cd901965e6 <unknown>
#18 0x7d080149caa4 <unknown>
#19 0x7d0801529c3c <unknown>



In [56]:
# extract_match_data(driver, match_hrefs)

# Repeat via Previous button
while True:
    try:
        prev_btn = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div/div[2]/div[1]/div/div/div/button")
        driver.execute_script("arguments[0].scrollIntoView(true);", prev_btn)
        time.sleep(0.5)  # Ensure visibility
        prev_btn.click()
        time.sleep(2)  # allow page to load/update

        extracted_data = extract_match_data(driver, match_hrefs)
        print(extracted_data)

    except Exception as e:
        print("No more previous matches or error occurred:", e)
        break

No more previous matches or error occurred: Message: element click intercepted: Element <button class="sc-41ba8c7a-0 dGqIGb undefined">...</button> is not clickable at point (518, 14). Other element would receive the click: <div class="fc-consent-root">...</div>
  (Session info: chrome=138.0.7204.183); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
#0 0x55cd901972ca <unknown>
#1 0x55cd8fc3e550 <unknown>
#2 0x55cd8fc96f5c <unknown>
#3 0x55cd8fc94dc2 <unknown>
#4 0x55cd8fc92462 <unknown>
#5 0x55cd8fc91b80 <unknown>
#6 0x55cd8fc842ea <unknown>
#7 0x55cd8fcb5bc2 <unknown>
#8 0x55cd8fc83c6a <unknown>
#9 0x55cd8fcb5d8e <unknown>
#10 0x55cd8fcdb9e6 <unknown>
#11 0x55cd8fcb5993 <unknown>
#12 0x55cd8fc81d6b <unknown>
#13 0x55cd8fc83141 <unknown>
#14 0x55cd9015c2ab <unknown>
#15 0x55cd901600b9 <unknown>
#16 0x55cd90143139 <unknown>
#17 0x55cd90160c68 <unknown>
#18 0x55cd9012760f <

In [42]:
driver.get(url)
prev_btn = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div/div[2]/div[1]/div/div/div/button")
print(prev_btn.text)

Load previous


In [72]:
[1,2,3] + [4,5,6]

[1, 2, 3, 4, 5, 6]

In [47]:
print(int(home_score.text.strip()) + int(away_score.text.strip()))

DEBUG:selenium.webdriver.remote.remote_connection:GET http://localhost:40069/session/9d086d71c326b352c92572dda3d12b61/element/f.01E220F94D55318FAFB30F863D80B09F.d.F2E77589189FA1545767DB1810D30491.e.10032/text {}
DEBUG:urllib3.connectionpool:http://localhost:40069 "GET /session/9d086d71c326b352c92572dda3d12b61/element/f.01E220F94D55318FAFB30F863D80B09F.d.F2E77589189FA1545767DB1810D30491.e.10032/text HTTP/1.1" 200 0
DEBUG:selenium.webdriver.remote.remote_connection:Remote response: status=200 | data={"value":"3"} | headers=HTTPHeaderDict({'Content-Length': '13', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:GET http://localhost:40069/session/9d086d71c326b352c92572dda3d12b61/element/f.01E220F94D55318FAFB30F863D80B09F.d.F2E77589189FA1545767DB1810D30491.e.10033/text {}
DEBUG:urllib3.connectionpool:http://localhost:40069 "GET /session/9d086d7

3


In [ ]:
# body > div.sc-99237a02-0.emYNNA:nth-child(5) > div.sc-81220147-0.jXeDBc > div.sc-81220147-1.eIIZxi.sc-19cad2e4-0.kkiDdB > div.sc-81220147-2.fgqGfg.simple-grid-item:nth-child(1) > div.sc-94e50ec4-0.dwozFa > div.sc-9ab61def-0.fHlmFM.sc-b4561063-12.hMGjRc:nth-child(1) > div.sc-b4561063-2.dgmodi > div.sc-9ab61def-0.cvqIJV.sc-b4561063-0.bItxxz:nth-child(3) > div.sc-b4561063-1.fzYiaa

# body > div.sc-99237a02-0.emYNNA:nth-child(5) > div.sc-81220147-0.jXeDBc > div.sc-81220147-1.eIIZxi.sc-19cad2e4-0.kkiDdB > div.sc-81220147-2.fgqGfg.simple-grid-item:nth-child(1) > div.sc-94e50ec4-0.dwozFa > div.sc-9ab61def-0.fHlmFM.sc-b4561063-12.hMGjRc:nth-child(1) > div.sc-b4561063-2.dgmodi > div.sc-9ab61def-0.cvqIJV.sc-b4561063-0.bItxxz:nth-child(3) > div.sc-b4561063-1.fzYiaa > div.sc-b4561063-8.lndvMZ:nth-child(2) > div.sc-b4561063-9.ckSzrE:nth-child(2) > div.sc-94e50ec4-0.ksqFna:nth-child(1) > div.sc-4e4c9eab-1.fPbPfi.label.sc-b4561063-6.hJImxc

# body > div.sc-99237a02-0.emYNNA:nth-child(5) > div.sc-81220147-0.jXeDBc > div.sc-81220147-1.eIIZxi.sc-19cad2e4-0.kkiDdB > div.sc-81220147-2.fgqGfg.simple-grid-item:nth-child(1) > div.sc-94e50ec4-0.dwozFa > div.sc-9ab61def-0.fHlmFM.sc-b4561063-12.hMGjRc:nth-child(1) > div.sc-b4561063-2.dgmodi > div.sc-9ab61def-0.cvqIJV.sc-b4561063-0.bItxxz:nth-child(3) > div.sc-b4561063-1.fzYiaa > div.sc-b4561063-8.lndvMZ:nth-child(2) > div.sc-b4561063-9.ckSzrE:nth-child(2) > div.sc-94e50ec4-0.ksqFna:nth-child(3) > div.sc-4e4c9eab-1.fPbPfi.label.sc-b4561063-6.hJImxc

# team   /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/a[1]/div
# team   /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/a[2]/div
#        /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/a[1]/div

# team_name   /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/a[1]/div/div/div/h1
# team_name   /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/a[2]/div/div/div/h1

# team_badge  /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/a[1]/div/span/img
# team_badge  /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/a[2]/div/span/img


# goals  /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div
# goals  /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[3]/div/div/div/div[3]/div


#        /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[5]/div[1]
#        /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[5]/div[2]


# /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[5]/div[1]/div[1]/span
# /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[5]/div[1]/div[2]/span
# /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[5]/div[2]/div/span


# /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[5]/div[1]/div[1]
# /html/body/div[3]/div/div/div[1]/div/div[1]/div/div[5]/div[1]

# events_div_xpath = "/html/body/div[3]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div[2]/div/div[2]/div[1]"